In [1]:
import re
import geopandas as gpd

In [2]:
import plotly_express as px
import dash
from dash import dcc
from dash import html

In [3]:
# Load your geodatabase
gdb_path = "../data/Asam_data_download.gdb"

df = gpd.read_file(gdb_path)

In [4]:
df.head(2)

,reference,dateofocc,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,year,yeartxt,...,nga_disseminationadditional,authority_l,navregion,entrydate,editor,authority_email,authority_phone,source_date,globalid,geometry
0,1990-9,1990-06-03 00:00:00+00:00,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2.0,11.0,1990,1990,...,None,2.0,NAVAREA IV,1990-07-12 00:00:00+00:00,3596,None,None,NaT,{90696A99-CA54-47EE-A881-45950BA39952},POINT (-75.13333 21.93333)
1,1990-10,1990-03-20 00:00:00+00:00,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1.0,3.0,1990,1990,...,None,2.0,HYDROPAC,1990-07-12 00:00:00+00:00,4144,None,None,NaT,{EC54E0FF-F01C-4B4D-9914-7A794F6BFC88},POINT (108 3)


In [5]:
df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 9182 entries, 0 to 9181
Data columns (total 30 columns):
 #   Column                       Non-Null Count  Dtype              
---  ------                       --------------  -----              
 0   reference                    9182 non-null   object             
 1   dateofocc                    9182 non-null   datetime64[ms, UTC]
 2   subreg                       9182 non-null   object             
 3   hostility_d                  8353 non-null   object             
 4   victim_d                     8767 non-null   object             
 5   description                  9176 non-null   object             
 6   hostilitytype_l              9157 non-null   float64            
 7   victim_l                     9155 non-null   float64            
 8   year                         9182 non-null   int16              
 9   yeartxt                      9182 non-null   object             
 10  deaths                       9182 non-nu

In [13]:
what_am_I_doing_with_this = list(df.select_dtypes(['object']).columns)

In [8]:
drop_cols = ['nga_releasability', 'nga_classification', 'nga_dissemination', 'nga_disseminationadditional',
                'navregion', 'editor', 'authority_email', 'authority_phone', 'globalid']

In [9]:
catagorical_cols = ['subreg', 'hostility_d', 'victim_d', 'hostilitytype_l', 'victim_l', 'authority_d', 'navarea', 'authority_l']

In [10]:
df

,reference,dateofocc,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,year,yeartxt,...,nga_disseminationadditional,authority_l,navregion,entrydate,editor,authority_email,authority_phone,source_date,globalid,geometry
0,1990-9,1990-06-03 00:00:00+00:00,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2.0,11.0,1990,1990,...,None,2.0,NAVAREA IV,1990-07-12 00:00:00+00:00,3596,None,None,NaT,{90696A99-CA54-47EE-A881-45950BA39952},POINT (-75.13333 21.93333)
1,1990-10,1990-03-20 00:00:00+00:00,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1.0,3.0,1990,1990,...,None,2.0,HYDROPAC,1990-07-12 00:00:00+00:00,4144,None,None,NaT,{EC54E0FF-F01C-4B4D-9914-7A794F6BFC88},POINT (108 3)
2,1990-11,1990-03-20 00:00:00+00:00,61,PIRATES,RO/RO SUNRISE,20 MARCH 1990. SINGAPORE. ...,1.0,3.0,1990,1990,...,None,2.0,HYDROPAC,1990-07-12 00:00:00+00:00,3703,None,None,NaT,{EA340298-971B-4948-97A9-FBB7DD11CDD2},POINT (90 -1)
3,1989-16,1989-01-01 00:00:00+00:00,62,PEOPLES DEMOCRATIC REPUBLIC OF YEMEN,U.S. MARINERS,"RED SEA, YEMEN ...",2.0,13.0,1989,1989,...,None,4.0,HYDROPAC,1989-08-31 00:00:00+00:00,3703,None,None,NaT,{5175409B-8939-42F8-A3E6-0FB837AE4935},POINT (42 14)
4,1989-17,1989-09-23 00:00:00+00:00,63,PIRATES,LASH STONEWALL JACKSON,Indian ocean ...,1.0,11.0,1989,1989,...,None,10.0,HYDROPAC,1989-10-02 00:00:00+00:00,3703,None,None,NaT,{62030092-38EC-4B1C-B712-01B159039F06},POINT (80.3 13.1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9177,2021-160,2021-02-09 13:54:58+00:00,57,None,None,"NIGERIA: On 9 February, pirates in speedboats ...",4.0,12.0,2021,2021,...,None,9.0,HYDROLANT,2021-07-23 13:58:14+00:00,1398693597,None,None,2021-02-24 13:55:06+00:00,{1B05CA6C-4D6C-46DE-BC74-D58736B00A95},POINT (6.2608 4.3748)
9178,2023-131,2023-07-23 22:20:29+00:00,71,None,Barge under tow by a tugboat,"MALAYSIA: On 23 July at 2220 local time, four ...",11.0,2.0,2023,2023,...,None,9.0,HYDROPAC,2023-09-08 12:47:38+00:00,1398693597,None,None,2023-07-26 12:45:50+00:00,{07493774-A3F6-4F15-B26B-7243E1F1F42D},POINT (104.16667 1.3)
9179,2024-52,2023-12-30 10:58:43+00:00,61,None,Fishing dhow AL SAQAR,"SOMALIA: On 30 December at 1058 UTC, the fishi...",7.0,4.0,2023,2023,...,None,9.0,HYDROPAC,2024-02-01 12:35:28+00:00,1398693597,None,None,2024-01-03 12:34:09+00:00,{717D4534-4E0B-4576-B700-3B91F3DC3117},POINT (51.29941 10.52682)
9180,2020-188,2020-01-17 13:39:03+00:00,57,Two robbers attempted boarding vessel,Tanker at anchor,"On 17 January, robbers in a small boat approac...",9.0,1.0,2020,2020,...,None,9.0,HYDROLANT,2020-05-13 13:13:10+00:00,1398693597,None,None,2020-02-13 13:39:00+00:00,{4FC08021-7CA4-4ECB-B45D-F4239887C139},POINT (3.196 6.315)


In [14]:
#df.info()

In [10]:
cols = ['subreg', 'hostility_d', 'victim_d', 'hostilitytype_l', 'victim_l', 'authority_d', 'navarea', 'authority_l']


def all_uniques_in_cols(df, cols=None):
    '''
    use: lists all the unique values for selected columns. Default is all columns
    input: dataframe, optional list of columns
    output: formatted output of all the unique values in the dataframe
    '''
    if cols is None:
        cols = df.columns.to_list()
    
    for col in cols:
        print(f"""There are {df[col].nunique()} unique values for {col}: 
              {df[col].unique()}
              """)

# Example usage
# all_uniques_in_all_cols(df) 
# all_uniques_in_all_cols(df, ['subreg','navregion']) 

In [12]:
all_uniques_in_cols(df, cols)

There are 40 unique values for subreg: 
              ['26' '71' '61' '62' '63' '24' '22' '92' '91' '93' '53' '57' '28' '25'
 '56' '51' '94' '52' '96' '72' '54' '21' '97' '73' '95' '55' '74' '37'
 '11' '12' '81' '82' '83' '44' '23' '36' '27' '18' '13' '35']
              
There are 528 unique values for hostility_d: 
              ['CUBAN GUNBOAT' 'PIRATES' 'PEOPLES DEMOCRATIC REPUBLIC OF YEMEN'
 'MOGADISHU REBELS' 'PIRATE' 'ASSAILANTS' 'THIEVES' 'STOWAWAYS' 'REBELS'
 'Pirates' 'Robbers' 'PIRATESS' 'UNKNOWN  TERRORISTS' 'SYRIA' 'UNKNOWN'
 'SYRIAN GUNBOATS' 'PIRATES/THIEVES' 'HIJACKERS' 'INTRUDERS' 'THUGS'
 'assailants' 'N/A' 'PIRATES, ARMED MEN' 'MOROCCAN NAVAL FORCES'
 'KAREN PIRATES' 'BANDITS' 'CHINESE WARSHIP' 'pirates' 'militiamen'
 'GUINEA-BISSAU GOVERNMENT FORCES' 'LTTE TERRORISTS'
 'CHINESE NAVY VESSEL NO. 162' 'apparent chinese customs officials'
 'NIGERIAN MILITARY FORCES' 'CHINESE BORDER VESSEL' 'Assailants'
 'PIRATES (PEOPLES REPUBLIC OF CHINA)' 'UNKNOWN PEOPLE'
 'UNIDENTIFI

In [11]:
def preprocess(df):
    """
    preprocesses csv by:
        Drop columns with no event description (critical issue)
        Drop columns where all values are NaN (i.e., 'Unnamed: 8')
        Change NaN values to 'No Information' (critical issue)
    """

    df.dropna(subset=["description"], inplace=True)
    df.dropna(axis=1, how="all", inplace=True)
    df.fillna("No information", inplace=True)

    return df

In [12]:
preprocess(df)

/var/folders/_3/9_qsgmm55yg0k3k8tgh0gy3w0000gn/T/ipykernel_67416/2137710158.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No information' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna("No information", inplace=True)
/var/folders/_3/9_qsgmm55yg0k3k8tgh0gy3w0000gn/T/ipykernel_67416/2137710158.py:12: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'No information' has dtype incompatible with datetime64[ms, UTC], please explicitly cast to a compatible dtype first.
  df.fillna("No information", inplace=True)


,reference,dateofocc,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,year,yeartxt,...,nga_dissemination,authority_l,navregion,entrydate,editor,authority_email,authority_phone,source_date,globalid,geometry
0,1990-9,1990-06-03 00:00:00+00:00,26,CUBAN GUNBOAT,BELESBAT QUEEN,A CUBAN GUNBOAT COMMANDEERED LUXURY YACHT BELE...,2.0,11.0,1990,1990,...,NONE,2.0,NAVAREA IV,1990-07-12 00:00:00+00:00,3596,No information,No information,No information,{90696A99-CA54-47EE-A881-45950BA39952},POINT (-75.13333 21.93333)
1,1990-10,1990-03-20 00:00:00+00:00,71,PIRATES,RO/RO SEA DRAGON,20 MARCH 1990. BORNEO. ...,1.0,3.0,1990,1990,...,NONE,2.0,HYDROPAC,1990-07-12 00:00:00+00:00,4144,No information,No information,No information,{EC54E0FF-F01C-4B4D-9914-7A794F6BFC88},POINT (108 3)
2,1990-11,1990-03-20 00:00:00+00:00,61,PIRATES,RO/RO SUNRISE,20 MARCH 1990. SINGAPORE. ...,1.0,3.0,1990,1990,...,NONE,2.0,HYDROPAC,1990-07-12 00:00:00+00:00,3703,No information,No information,No information,{EA340298-971B-4948-97A9-FBB7DD11CDD2},POINT (90 -1)
3,1989-16,1989-01-01 00:00:00+00:00,62,PEOPLES DEMOCRATIC REPUBLIC OF YEMEN,U.S. MARINERS,"RED SEA, YEMEN ...",2.0,13.0,1989,1989,...,NONE,4.0,HYDROPAC,1989-08-31 00:00:00+00:00,3703,No information,No information,No information,{5175409B-8939-42F8-A3E6-0FB837AE4935},POINT (42 14)
4,1989-17,1989-09-23 00:00:00+00:00,63,PIRATES,LASH STONEWALL JACKSON,Indian ocean ...,1.0,11.0,1989,1989,...,NONE,10.0,HYDROPAC,1989-10-02 00:00:00+00:00,3703,No information,No information,No information,{62030092-38EC-4B1C-B712-01B159039F06},POINT (80.3 13.1)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9177,2021-160,2021-02-09 13:54:58+00:00,57,No information,No information,"NIGERIA: On 9 February, pirates in speedboats ...",4.0,12.0,2021,2021,...,NONE,9.0,HYDROLANT,2021-07-23 13:58:14+00:00,1398693597,No information,No information,2021-02-24 13:55:06+00:00,{1B05CA6C-4D6C-46DE-BC74-D58736B00A95},POINT (6.2608 4.3748)
9178,2023-131,2023-07-23 22:20:29+00:00,71,No information,Barge under tow by a tugboat,"MALAYSIA: On 23 July at 2220 local time, four ...",11.0,2.0,2023,2023,...,NONE,9.0,HYDROPAC,2023-09-08 12:47:38+00:00,1398693597,No information,No information,2023-07-26 12:45:50+00:00,{07493774-A3F6-4F15-B26B-7243E1F1F42D},POINT (104.16667 1.3)
9179,2024-52,2023-12-30 10:58:43+00:00,61,No information,Fishing dhow AL SAQAR,"SOMALIA: On 30 December at 1058 UTC, the fishi...",7.0,4.0,2023,2023,...,NONE,9.0,HYDROPAC,2024-02-01 12:35:28+00:00,1398693597,No information,No information,2024-01-03 12:34:09+00:00,{717D4534-4E0B-4576-B700-3B91F3DC3117},POINT (51.29941 10.52682)
9180,2020-188,2020-01-17 13:39:03+00:00,57,Two robbers attempted boarding vessel,Tanker at anchor,"On 17 January, robbers in a small boat approac...",9.0,1.0,2020,2020,...,NONE,9.0,HYDROLANT,2020-05-13 13:13:10+00:00,1398693597,No information,No information,2020-02-13 13:39:00+00:00,{4FC08021-7CA4-4ECB-B45D-F4239887C139},POINT (3.196 6.315)


In [ ]:

def split_gdb_LatLong(df):
    """
    Converts the 'position' column of an esri geodatabase (.gdb) compliant column into separate columns ('latitude' and 'longitude')
    Removes  the 'position' column.
    """

    df = df.join(df["position"].str.split(expand=True)).rename(
        columns={0: "latitude", 1: "longitude"}
    )
    df.drop("position", axis=1, inplace=True)

    return df


def baseline_processing(df):
    preprocess(df)
    df = splitLatLong(df)
    df = conv_to_dd(df)
    print("Preprocessing completed")

    return df







def export_as_pickle(df):
    df.to_pickle("../data/in_progress/pickled_data.pkl")



#### 431 types of hostilities, and 1587 types of victims both seem a bit ... excessive.
And it's not like it can be argued these are the total number of cases because there are 8619 events. This isn't critical for now though so let's move on.

### Checking for additional suprizes

In [13]:
df['navarea'].unique()

array(['IV', 'XI', 'VIII', 'IX', 'V', 'XII', 'III', 'II', 'VII', 'XIII',
       None, 'XVI', 'X', 'XV', 'I', 'XIV', 'VI'], dtype=object)

In [17]:
# Not sure why this is, it says there are some Nones in the columns

df.loc[df['navarea'] == None]

,reference,dateofocc,subreg,hostility_d,victim_d,description,hostilitytype_l,victim_l,year,yeartxt,...,nga_disseminationadditional,authority_l,navregion,entrydate,editor,authority_email,authority_phone,source_date,globalid,geometry


The entries have lat/long values so I'm not worried about it and can correct this in the future. I'm curious about why it wasn't entered in the first place though. 
<br><br>
Subreg has no issues as you can see below.

In [18]:
df['subreg'].unique()

array(['26', '71', '61', '62', '63', '24', '22', '92', '91', '93', '53',
       '57', '28', '25', '56', '51', '94', '52', '96', '72', '54', '21',
       '97', '73', '95', '55', '74', '37', '11', '12', '81', '82', '83',
       '44', '23', '36', '27', '18', '13', '35'], dtype=object)

## No graphing possible until I get the lat/lon thing fixes

In [ ]:
fig = px.scatter_mapbox( data_frame=df, lat="latitude", lon="longitude",
        mapbox_style="carto-darkmatter", color_discrete_sequence=["gold"], 
        zoom=1, height=700, width=1500, center= {'lat': 16 ,'lon': 99})

fig.show()

I like the contrast but this is still isn't as good as the USG sat maps. They show water depth and indirectly the temp with the poles being white.

In [22]:
years = list({i for i in df['year']})


Redoing color was for each year below was tricky but I like how terse but readable it is

In [ ]:
fig = px.scatter_mapbox(data_frame=df, lat="latitude", lon="longitude",
        mapbox_style="carto-darkmatter", color=df['date'].dt.year, 
        zoom=1, height=700, width=1500, center= {'lat': 16 ,'lon': 99})

fig.show()